In [1]:
%run ./../../utils/commonImports.py
%matplotlib inline

# Load Data

In [2]:
db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data = load_trading_data(db, 'chart_data', from_date='2014-01-01', period='5min')

In [3]:
data.head()

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date
0,1424373000,5min,225.0000,225.0000,225.0000,225.0000,1.0000,0.0044,225.0000,2015-02-19 19:10:00
1,1424373300,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:15:00
2,1424373600,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:20:00
3,1424373900,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:25:00
4,1424374200,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:30:00


# Test Data - Timestamp diff

In [4]:
diffs = data['timestamp'].shift(-1) - data['timestamp']
data['timestampDiff'] = diffs

In [5]:
diffs.value_counts()

300.0000    381241
Name: timestamp, dtype: int64

In [7]:
zero_diff = data.query('timestampDiff == 1200')
zero_diff

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date,timestampDiff


# Day Difference Calc

In [15]:
from datetime import timedelta

def day_diff(ohlc):
    diffs = []
    for i in range(ohlc.shape[0]):
        row = ohlc.iloc[i, :]
        current_date = row['day']
        yesterday_date = current_date - timedelta(days=1)

In [30]:
d = {}
d.get('5', 0)

0

In [31]:
from utils.date import timestamp_to_date
from datetime import timedelta


def ohlc4(day_data):
    day_data = day_data.reset_index()
    _open = day_data.loc[0, 'open']
    close = day_data.iloc[-1, :]['close']
    high = day_data['high'].max()
    low = day_data['low'].min()
    return np.mean([_open, close, high, low])
    

ohlc = data[['timestamp', 'period', 'open', 'high', 'low', 'close', 'date']].copy()
ohlc['day'] = ohlc['timestamp'].apply(timestamp_to_date).apply(lambda x: x.date())
ohlc['yesterday'] = ohlc['day'] - timedelta(days=1)

diffs = []
yesterday_means = {}
for i in range(ohlc.shape[0]):
    if i % 1000 == 999:
        print(f"{i}/{ohlc.shape[0]}")
    
    row = ohlc.iloc[i, :]
    yesterday = ohlc[ohlc['day'] == row['yesterday']]
    if yesterday.shape[0] == 0:
        diffs.append(-10)
        continue
    
    today = ohlc.iloc[:i+1, :]
    today = today[today['day'] == row['day']]
    yesterday_mean = yesterday_means.get(row['yesterday'], ohlc4(yesterday))
    yesterday_means[row['yesterday']] = yesterday_mean
    today_mean = ohlc4(today)
    diffs.append((today_mean - yesterday_mean) / yesterday_mean)
    
data['dayDiff'] = diffs

999/381242


KeyboardInterrupt: 

In [21]:
ohlc

,timestamp,period,open,high,low,close,date,day,yesterday
0,1424373000,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:10:00,2015-02-19,2015-02-18
1,1424373300,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:15:00,2015-02-19,2015-02-18
2,1424373600,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:20:00,2015-02-19,2015-02-18
3,1424373900,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:25:00,2015-02-19,2015-02-18
4,1424374200,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:30:00,2015-02-19,2015-02-18
5,1424374500,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:35:00,2015-02-19,2015-02-18
6,1424374800,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:40:00,2015-02-19,2015-02-18
7,1424375100,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:45:00,2015-02-19,2015-02-18
8,1424375400,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:50:00,2015-02-19,2015-02-18
9,1424375700,5min,225.0000,225.0000,225.0000,225.0000,2015-02-19 19:55:00,2015-02-19,2015-02-18
